<a href="https://colab.research.google.com/github/pavankumarm2505/Intro_to_DL/blob/main/Copy_of_Assignment_1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - 0001
1. Build your first Neural Network and test it on MNIST and F-MNIST/K-MNIST
2. No - Plagarism. If solutions match, then the student who shared the solution and students who copied will be reported to USF and will get F in this course.
3. You can discuss, but have to code and provide your own solution. Provide name of the person or persons with whom you had discussion

## Things to Do
 
1. Create a Neural Network with 3 layers, (W1, W2, W3)
2. H1 = $X \cdot W1$ + b1, Z1 = $\sigma$(H1), where $\sigma$ is your activation function, $\cdot$ is your hardamard product, H denotes pre-activation and Z denotes post-activation.
3. Recursively construct your NN, H2 = (Z1 $\cdot$W2) + b2, Z2 = $\sigma$(H2) and final output layer is H3 = (Z2 $\cdot$ W3) + b3, z3 = $σ^1$(H3), where $σ^1$ is your softmax function
4. Normalize your data within 0-1 range, create one-hot encoding for your labels.
5. Take 3digit of your first name, and convert them based on character indices - For example: Ank ---> 1 14 11, so my seed is 11411. This will ensure each students have unique seeds.
6. Run your experiments for minimum $10$ epochs. Remember one epoch is entire pass through data.
7. You will run each experiment min $5$ times and report your average accuracy and standard error -- Change seed per trial
8. We have provided 2 folders, members of Teams A - E will work on data that says Team_red and Team F- K will work on data that says Team_blue
9. Important: All parameter optimization should be done on Validation/Dev Set, you will only do one pass inference over test set.
10. Complete all code blocks highlighted as TODO
11. Sumbit your code, your report with findings and upload on canvas.

## Rubric
1. Total marks = 10% of your total grade. 
2. This assignment is of worth 100 points, which will be converted into 10.
3. If you train on Vaidation and test sets ---- (0 or -100 Points)
4. Results with only one trial is reported ---- (-20 points)
5. Experiments are performed on only one dataset ---- (-50 points)
6. Code is not working or has some bugs, Depending on bug ---- (-15 to -60 points)
7. Submiting some random solution without following template ---- (-100 points)

In [72]:
import os
import numpy as np
import time
import tensorflow as tf
from sklearn import preprocessing

seed = 16122
np.random.seed(seed) ## Use your own unique seed
tf.random.set_seed(seed) ## Use your own unique seed

## Authors - Ankur Mali
## Code designed for intro to DL at University of South Florida

In [73]:
## Define model specific parameters
## Repalce None with values
size_input = 28*28
size_hidden = 256
size_hidden1 = 256
size_hidden2 = 256
size_hidden3 = 256
size_output = 10
number_of_train_examples = 50000
number_of_dev_examples = 10000
number_of_test_examples = 10000
batch_size = 32
NUM_EPOCHS = 10

In [74]:
## TODO
## Load Data
X_train = np.load('X_train.npy').reshape(-1,784)
print(X_train.shape)
X_test = np.load('X_test.npy').reshape(-1,784)
## Create Validation data from train splits

Y_train = np.load('y_train.npy')
Y_test = np.load('y_test.npy')
X_val, Y_val = X_train[50000:], Y_train[50000:] 
X_train,Y_train = X_train[:50000], Y_train[:50000]



(60000, 784)


In [75]:
## TODO
# Normalize your X
X_train = (X_train - np.mean(X_train,axis=0)) / np.std(X_train,axis=0)
X_val = (X_val - np.mean(X_val,axis=0)) / np.std(X_val,axis=0)
X_test = (X_test - np.mean(X_test,axis=0)) / np.std(X_test,axis=0)

# Convert your labels into one-hot encoding

Y_train = tf.keras.utils.to_categorical(Y_train, size_output)
Y_val = tf.keras.utils.to_categorical(Y_val,size_output)
Y_test = tf.keras.utils.to_categorical(Y_test,size_output)
# Hint look into function tf.keras.utils.to_categorical or Other function is tf.one_hot()




In [76]:
# Define class to build mlp model
class MLP(object):
 def __init__(self, size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of the 1st hidden layer
    size_hidden2: int, size of the 2nd hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_hidden3, self.size_output, self.device =\
    size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device
    
    ### TODO
    ### Declare all your variables
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1]))
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2]))
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_output]))

    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden1]))
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden2]))
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
  
 def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y

 def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    ## TODO

    # Convert your pred and true to tf.float32
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)


    # Ensure your shapes are (batch_size, size_output)
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # This above function will add softmax to your final layer and then perform cross-entropy
    loss_x = cce(y_true_tf, y_pred_tf)
    
    
    return loss_x

 def backward(self, X_train, y_train):
    """
    backward pass
    """
    lr = 0.1 #Play with your learning rate
    optimizer = tf.keras.optimizers.SGD(learning_rate = lr) # For this excercise we will only use SGD
    
    with tf.GradientTape() as tape:
        
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
        
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables)) # Optimizer
           
 def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)

    ## TODO
    H1 = tf.matmul(X_tf, self.W1) + self.b1
    Z1 = tf.nn.sigmoid(H1)
    H2 = tf.matmul(Z1, self.W2) + self.b2
    Z2 = tf.nn.sigmoid(H2)
    H3 = tf.matmul(Z2, self.W3) + self.b3
    
    
    # Compute values in hidden layers
    

    # Compute output
    output = H3 # Logits
    
    # Remember the loss function has keras loss objective, that explicitly applies softmax over logits
    return (output)

In [77]:
# Initialize model using CPU
mlp_on_gpu = MLP(size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device='gpu')

time_start = time.time()

for epoch in range(NUM_EPOCHS):
    
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
    
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(25, seed=epoch*(seed)).batch(batch_size)
  kz = 0
  accuracy_z = 0.0
  cur_train_acc = 0.0
  accuracy_v = 0.0
  for inputs, outputs in train_ds:
    qw, tr = tf.shape(inputs)
    kz = kz + 1
    preds = mlp_on_gpu.forward(inputs) 
    loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
    lt = lt + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)

  preds = mlp_on_gpu.forward(X_train)
  # Get probs, remember we only have logits from our forward function, we need to apply softmax on top of it to get probs
  preds = tf.nn.softmax(preds)
  correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_train, 1))
  accuracy_z = accuracy_z + tf.reduce_mean(tf.cast(correct_prediction, "float"))
  cur_train_acc += accuracy_z.numpy()
  ds = cur_train_acc
  print('\nTrain Accuracy: {:.4f}'.format(ds))
  print('Number of Epoch = {} - Average Cross Entropy:= {} '.format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
  
  ## TODO

  ## Now calculate Validation Accuracy

  preds = mlp_on_gpu.forward(X_val)
  preds = tf.nn.softmax(preds)
  correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_val, 1))
  accuracy_v =  tf.reduce_mean(tf.cast(correct_prediction, "float"))
  cur_val_acc = accuracy_v.numpy()


  print('\nValidation Accuracy: {:.4f}'.format(cur_val_acc))
  


        
time_taken = time.time() - time_start
    
# Validate model
    


print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs


Train Accuracy: 0.7747
Number of Epoch = 1 - Average Cross Entropy:= 0.0496722314453125 

Validation Accuracy: 0.7384

Train Accuracy: 0.8150
Number of Epoch = 2 - Average Cross Entropy:= 0.02460473388671875 

Validation Accuracy: 0.7630

Train Accuracy: 0.8350
Number of Epoch = 3 - Average Cross Entropy:= 0.018439703369140626 

Validation Accuracy: 0.7756

Train Accuracy: 0.8434
Number of Epoch = 4 - Average Cross Entropy:= 0.015484481201171876 

Validation Accuracy: 0.7699

Train Accuracy: 0.8601
Number of Epoch = 5 - Average Cross Entropy:= 0.013615498046875 

Validation Accuracy: 0.7796

Train Accuracy: 0.8699
Number of Epoch = 6 - Average Cross Entropy:= 0.012320548095703125 

Validation Accuracy: 0.7834

Train Accuracy: 0.8669
Number of Epoch = 7 - Average Cross Entropy:= 0.011348411865234375 

Validation Accuracy: 0.7726

Train Accuracy: 0.8901
Number of Epoch = 8 - Average Cross Entropy:= 0.010570909423828125 

Validation Accuracy: 0.7879

Train Accuracy: 0.8990
Number of Epoc

In [84]:
## TODO - Write a inference code and perform only one pass through test data
test_loss_total = tf.Variable(0, dtype=tf.float32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(4)
#test_loss_total = 0.0
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  preds = tf.nn.softmax(preds)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)

print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_test.shape[0]))


Test MSE: 0.4244
